In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import re

In [2]:
OutPathCSV = 'OUTPUT/filtered-companies.csv'
OutPathJSON = 'OUTPUT/filtered-companies.json'

In [3]:
dbName="companies"
mongodbURL = f"mongodb://localhost/{dbName}"

# access database and avoid timeouts
mclient = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = mclient.get_database()

In [4]:
cur = db.list_collections()

In [5]:
for col in cur:
    print(col)

{'name': 'companies', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('76992dc7-b5e1-4dcd-9a37-b36c0e165615')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'companies.companies'}}


In [6]:
# query to only bring certain attributes from the complete collection
# relevant data:
# - offices
# - name
# - category_code
# - founded_year
# - succesful startups that raised more then 1M USD or 750K EUR

query = {"founded_year":{"$gt":2009} # Only young companies
        }
projection = {"offices":1,
              "name":1,
              "category_code":1,
              "founded_year":1,
              "acquisition":1,
              "funding_rounds":1, # an array of documents with 10 nested fields
              "total_money_raised":1 # a bunch of strings that need regex
             }

all_offices = list(db.companies.find(query,projection))

In [7]:
from pandas.io.json import json_normalize
all_offices = pd.DataFrame(json_normalize(all_offices))
all_offices.head()

/home/rh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,_id,name,category_code,founded_year,total_money_raised,funding_rounds,acquisition,offices,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,52cdef7c4bab8bd675297ea7,Mokitown,web,2011,$0,[],NaN,"[{'description': None, 'address1': None, 'addr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297f78,CircleUp,finance,2011,$9M,"[{'id': 29386, 'round_code': 'seed', 'source_u...",NaN,"[{'description': '', 'address1': '', 'address2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297f94,PeekYou,search,2012,$1.83M,"[{'id': 631, 'round_code': 'angel', 'source_ur...",NaN,"[{'description': None, 'address1': None, 'addr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52cdef7c4bab8bd675297f9e,GENWI,mobile,2010,$7.1M,"[{'id': 19988, 'round_code': 'seed', 'source_u...",NaN,"[{'description': '', 'address1': '4966 El Cami...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52cdef7c4bab8bd675297fb1,headr,web,2012,$0,[],NaN,"[{'description': '', 'address1': 'Ritterstr. 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Use .explode() to turn "offices" column, which is a series with a list of dictionaries,
# into to multiple rows.
# Each row represents an office location

companydata = pd.DataFrame(all_offices).explode("offices")
companydata.head(3)

,_id,name,category_code,founded_year,total_money_raised,funding_rounds,acquisition,offices,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,52cdef7c4bab8bd675297ea7,Mokitown,web,2011,$0,[],NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297f78,CircleUp,finance,2011,$9M,"[{'id': 29386, 'round_code': 'seed', 'source_u...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297f94,PeekYou,search,2012,$1.83M,"[{'id': 631, 'round_code': 'angel', 'source_ur...",NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Doing the .explode() to get info from the `funding_rounds` column
# which is an array of documents
companydata= pd.DataFrame(companydata).explode("funding_rounds")
companydata.head()

,_id,name,category_code,founded_year,total_money_raised,funding_rounds,acquisition,offices,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,52cdef7c4bab8bd675297ea7,Mokitown,web,2011,$0,NaN,NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297f78,CircleUp,finance,2011,$9M,"{'id': 29386, 'round_code': 'seed', 'source_ur...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297f78,CircleUp,finance,2011,$9M,"{'id': 37566, 'round_code': 'a', 'source_url':...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297f94,PeekYou,search,2012,$1.83M,"{'id': 631, 'round_code': 'angel', 'source_url...",NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297f94,PeekYou,search,2012,$1.83M,"{'id': 24725, 'round_code': 'angel', 'source_u...",NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Obtaining `GeoPoints`

In [10]:
# Transform the office column, as it holds objects (dictionary/json like) with
# GeoCoords for some of them. We want to turn that data into a "GeoPoint" format

# To do this, define a function that we will APPLY to the column with the data we need

# INPUT:
# - A pandas row from a dataframe with geo
# OUTPUT:
# - A Tupple with:
#     -`GeoPoint` or `None`
#     - A string with an error message

# This function will process the values in that row, and return from it a

def officeToGeoPoint(row):
    
    # Get the column with the object
    office= row.offices 
    
    # confirm that the variable contains a type() == dict:
    if type (office) == dict:
        
        # Confirm that the data is available within the dict
        if 'latitude' in office and 'longitude' in office:
            
            # Turn the Float values in the specified keys into GeoPoints:
            if (type(office['latitude'])) == float and type(office['longitude']):
                return ({
                    'type': 'Point',
                    'coordinates':[
                        office['longitude'],
                        office['latitude']
                    ]
                },
                "GeoPoint Found")
            
            # Explain possible errors 
            else:
                return (None, "Lat or Lon are not Floats")
        else:
            return(None, "keys 'latitude' and 'longitude' weren't found.")
    else:
        return (None, "No office object found for this row")
    

In [11]:
# Apply the function to each row (kwarg `axis=1`)
#
cleaned_offices = companydata.apply(officeToGeoPoint, # The function we made
                            axis=1,     # how to apply the function                
                            result_type="expand")

# The result_type='expand' will take
# the tupple returned from out officeToGeoPoint function
# and turns each value into columns.
# We will end up with a pandas DataFrame with
#   the same number of rows
#   but only two columns

In [12]:
#Assign a name to the columns
cleaned_offices.columns = ["GeoPoint_offices","GeoPoint_status"]
cleaned_offices.tail(5)

,GeoPoint_offices,GeoPoint_status
144,None,Lat or Lon are not Floats
144,None,Lat or Lon are not Floats
144,None,Lat or Lon are not Floats
144,None,Lat or Lon are not Floats
145,None,Lat or Lon are not Floats


In [13]:
cleaned_offices.GeoPoint_status.value_counts()

GeoPoint Found                         136
Lat or Lon are not Floats               89
No office object found for this row     32
Name: GeoPoint_status, dtype: int64

In [14]:
# Concatenate two Pandas Data Frames through their index.
# From this ' =  = '  
# to this   '  ==  '
companyGeoPointed = pd.concat([companydata, cleaned_offices], axis =1)
companyGeoPointed

,_id,name,category_code,founded_year,total_money_raised,funding_rounds,acquisition,offices,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink,GeoPoint_offices,GeoPoint_status
0,52cdef7c4bab8bd675297ea7,Mokitown,web,2011,$0,NaN,NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-95.712891, ...",GeoPoint Found
1,52cdef7c4bab8bd675297f78,CircleUp,finance,2011,$9M,"{'id': 29386, 'round_code': 'seed', 'source_ur...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Lat or Lon are not Floats
1,52cdef7c4bab8bd675297f78,CircleUp,finance,2011,$9M,"{'id': 37566, 'round_code': 'a', 'source_url':...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Lat or Lon are not Floats
2,52cdef7c4bab8bd675297f94,PeekYou,search,2012,$1.83M,"{'id': 631, 'round_code': 'angel', 'source_url...",NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-73.985506, ...",GeoPoint Found
2,52cdef7c4bab8bd675297f94,PeekYou,search,2012,$1.83M,"{'id': 24725, 'round_code': 'angel', 'source_u...",NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-73.985506, ...",GeoPoint Found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,52cdef7f4bab8bd67529c6d4,Treehouse,education,2011,$19.4M,"{'id': 25988, 'round_code': 'seed', 'source_ur...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Lat or Lon are not Floats
144,52cdef7f4bab8bd67529c6d4,Treehouse,education,2011,$19.4M,"{'id': 29379, 'round_code': 'unattributed', 's...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Lat or Lon are not Floats
144,52cdef7f4bab8bd67529c6d4,Treehouse,education,2011,$19.4M,"{'id': 36911, 'round_code': 'b', 'source_url':...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Lat or Lon are not Floats
144,52cdef7f4bab8bd67529c6d4,Treehouse,education,2011,$19.4M,"{'id': 55417, 'round_code': 'partial', 'source...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Lat or Lon are not Floats


In [15]:
companyGeoPointed.to_csv(OutPathCSV)

# Checkpoint, 
We already picked only the young companies from the dataset.
We also saved the basic dataset, now we can start more scoped data exploration

First, I want to see the companies which made at least 1 million USD or 750 k Euros,
I'll also consider the companies that were acquired for over that ammount.

In [16]:
df = companyGeoPointed
df.columns

Index(['_id', 'name', 'category_code', 'founded_year', 'total_money_raised',
       'funding_rounds', 'acquisition', 'offices', 'acquisition.price_amount',
       'acquisition.price_currency_code', 'acquisition.term_code',
       'acquisition.source_url', 'acquisition.source_description',
       'acquisition.acquired_year', 'acquisition.acquired_month',
       'acquisition.acquired_day', 'acquisition.acquiring_company.name',
       'acquisition.acquiring_company.permalink', 'GeoPoint_offices',
       'GeoPoint_status'],
      dtype='object')

In [17]:
df['acquisition.price_amount'].value_counts()
# This column was not very informative,
# But we can look at other objects in the collection

180000000.0    1
Name: acquisition.price_amount, dtype: int64

In [18]:
# Looking at the raised_amount with a regex
# I want to practice this a bit

searchRegEx = re.compile("M")
#fn = lambda money_string : money_string if "M" in money_string else "no millions"

# Catch millions and thousands
def money_counter(money_string):
    if ("M" in money_string) or ("k" in money_string):
        
        # I want all the millions
        if "M" in money_string:
            return money_string 
        # But only some of the hundreds
        elif "$" in money_string:
            return "little money"
        elif len(money_string) >= 4:
            return money_string
        else:
            return "WTF? - Error in money counter"
            
    else:
        return "little money"

In [19]:
df['total_money_raised'] = df.total_money_raised.apply(money_counter)
df.head(3)

,_id,name,category_code,founded_year,total_money_raised,funding_rounds,acquisition,offices,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink,GeoPoint_offices,GeoPoint_status
0,52cdef7c4bab8bd675297ea7,Mokitown,web,2011,little money,NaN,NaN,"{'description': None, 'address1': None, 'addre...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-95.712891, ...",GeoPoint Found
1,52cdef7c4bab8bd675297f78,CircleUp,finance,2011,$9M,"{'id': 29386, 'round_code': 'seed', 'source_ur...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Lat or Lon are not Floats
1,52cdef7c4bab8bd675297f78,CircleUp,finance,2011,$9M,"{'id': 37566, 'round_code': 'a', 'source_url':...",NaN,"{'description': '', 'address1': '', 'address2'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Lat or Lon are not Floats


In [23]:
# At this point, if we want to save the dataframe into a JSON
# the `_id` has to be dropped in order to avoid OverflowError

df2 = df.drop(columns="_id")
df2.columns

Index(['name', 'category_code', 'founded_year', 'total_money_raised',
       'funding_rounds', 'acquisition', 'offices', 'acquisition.price_amount',
       'acquisition.price_currency_code', 'acquisition.term_code',
       'acquisition.source_url', 'acquisition.source_description',
       'acquisition.acquired_year', 'acquisition.acquired_month',
       'acquisition.acquired_day', 'acquisition.acquiring_company.name',
       'acquisition.acquiring_company.permalink', 'GeoPoint_offices',
       'GeoPoint_status'],
      dtype='object')

In [25]:
# Export as json to do mongodb import
df2.to_json(OutPathJSON, orient='records')

# To do the MongoDB import trough bashline
# $ mongoimport --db datamad0320 --collection companies_prepared --jsonArray companies_clean.json

In [ ]:
####### Checkpoint
0/0

In [ ]:
companydata.to_csv(OutPathCSV)


In [ ]:
pd.json_normalize(OutPathCSV)
